In [31]:
#import Bio.PDB
#import Bio.SVDSuperimposer
#Я хотел переписать это через тот набор функций bio, который позволил бы мне посчитать RMS до преобразования.
#Там есть соответствующий пакет, именуемый Bio.SVDSuperimposer
#http://biopython.org/DIST/docs/api/Bio.SVDSuperimposer-module.html
import Bio.PDB.PDBParser
import Bio.SVDSuperimposer
#import Bio.PDB.Polypeptide
import numpy as np

pdb_code = "1JOY"
pdb_filename = "%s.pdb" % pdb_code
pdb_out_filename = "%s_aligned.pdb" % pdb_code

#Пока оставил, вообще говоря можно убрать (и вообще сократить код раза в три-четыре) 
#и идти тупо по всем атомам. Вообще по всем.
#seq_str = 'MAAGVKQLADDRTLLMAGVSHDLRTPLTRIRLATEMMSEQDGYLAESINKDIEECNAIIEQFIDYLR'
#use_str = 'MA---------RTLLMAGVSHDLRTPLTRIRLATEMMSEQDGYLAESINKDI---------------'
#Генерируем массив с единицами для совпадающих остатков
#use = [(letter != "-") for letter in use_str]
#assert len(use) == len(seq_str)


print ("Loading PDB file %s" % pdb_filename)
structure = Bio.PDB.PDBParser().get_structure(pdb_code, pdb_filename)

#print(structure[0])

Loading PDB file 1JOY.pdb


In [32]:
#Выведем RMSD по всем парам до выравнивания
#(Объяснение работы этого участка - ниже)
RMSD_start = 0.0;
for ref_model in structure :
    for alt_model in structure :
        ref_atoms = []
        alt_atoms = []
        for (ref_chain, alt_chain) in zip(ref_model, alt_model) :
            for ref_res, alt_res in zip(ref_chain, alt_chain) :
                assert ref_res.resname == alt_res.resname
                assert ref_res.id      == alt_res.id
                ref_atoms.append(ref_res['CA'].get_coord())
                alt_atoms.append(alt_res['CA'].get_coord())
        sup = Bio.SVDSuperimposer.SVDSuperimposer()
        ref_atoms = np.asarray(ref_atoms)
        alt_atoms = np.asarray(alt_atoms)
        #sup.set(ref_atoms, alt_atoms)
        RMSD_start += sup._rms(ref_atoms, alt_atoms)

In [33]:
print ("Everything aligned to first model...")
#Мы реализуем попарное сравнение, так что просто сравниваем все структуры с первой, выбранной статичной.
ref_model = structure[0]
#Идем по всем структурам:
for alt_model in structure :
    #Мы сравниваем не со всеми атомами, а с атомами альфа-углерода (?) в аминокислотах. 'CA' указывает на этот атом в
    #перебираемых аминокислотах
    ref_atoms = []
    alt_atoms = []
    #for (ref_chain, alt_chain) in zip(ref_model, alt_model) :
    #    print(alt_chain)
    #print(alt_model)
    #print(ref_model)
    for (ref_chain, alt_chain) in zip(ref_model, alt_model) :
        #for ref_res, alt_res, amino, allow in zip(ref_chain, alt_chain, seq_str, use) :
        for ref_res, alt_res in zip(ref_chain, alt_chain) :
            #Проверяем что это амин. одноо типа
            assert ref_res.resname == alt_res.resname
            #Проверяем, что они так же совпадают по месту в цепи 
            assert ref_res.id      == alt_res.id
            #Сверяем с шаблоном
            #assert amino == Bio.PDB.Polypeptide.three_to_one(ref_res.resname)
            #И, дополнительно, проверяем что мы хотим его сравнить. Последние 2 наверное в дроп пойдут.
            #if allow :
                #CA = alpha carbon
                #Добавляем координаты альфа-углерода у каждой из структур в список
            ref_atoms.append(ref_res['CA'].get_coord())
            alt_atoms.append(alt_res['CA'].get_coord())
                
    #Теперь выравниваем эти листы:
    #Инициализируем суперимпозер
    sup = Bio.SVDSuperimposer.SVDSuperimposer()
    #Листы превращаем в массивы
    ref_atoms = np.asarray(ref_atoms)
    alt_atoms = np.asarray(alt_atoms)
    #Инициализируем наш суперимпозер двумя наборами данных
    sup.set(ref_atoms, alt_atoms)

    #Какую пару сравниваем
    print ("model 1/model %i" % (alt_model.id+1))
    #RMSD для этой пары до сравнивания
    print ("RMS(bef) = %0.2f" % (sup._rms(ref_atoms, alt_atoms)))
    #Собственно выравниваем. При этом появляется матрица поворота и вектор сдвига - их можно применить ко всем атомам
    #второй структуры (хотя нам это особо не нужно)
    sup.run()

    #Выводим RMSD этой пары после выравнивания
    print ("RMS(aft) = %0.2f" % (sup.get_rms()))
    #Выводим матрицу поворота и вектор сдвига
    print (sup.get_rotran()[0], sup.get_rotran()[1])
    
    #Собственно преобразовываем координаты. Вроде это делается так, но так как данных много, 
    #то вообще говоря надо проверить. В иделае - визуализацией
    for res_atom in alt_model.get_atoms():
        res_atom.transform(sup.get_rotran()[0],sup.get_rotran()[1])
    
    #structure[alt_model] = alt_model

Everything aligned to first model...
model 1/model 1
RMS(bef) = 0.00
RMS(aft) = 0.00
[[  1.00000000e+00   0.00000000e+00  -2.04890966e-08]
 [  0.00000000e+00   1.00000000e+00   7.45058060e-09]
 [ -2.04890966e-08   7.45058060e-09   1.00000000e+00]] [ 0.  0.  0.]
model 1/model 2
RMS(bef) = 30.25
RMS(aft) = 2.03
[[-0.07667555 -0.80727816  0.58516908]
 [ 0.52585942  0.46589079  0.71163028]
 [-0.84710848  0.36228132  0.38879243]] [-16.72921181  -3.59168434  -5.46626949]
model 1/model 3
RMS(bef) = 23.08
RMS(aft) = 2.86
[[-0.41386262  0.51690161 -0.74935341]
 [-0.58216071 -0.78311747 -0.21866867]
 [-0.69986188  0.34574533  0.62502277]] [  6.55145645  10.84421539   8.77009678]
model 1/model 4
RMS(bef) = 40.62
RMS(aft) = 3.45
[[-0.08711918 -0.14012474 -0.98629367]
 [-0.20195454  0.97198433 -0.12025315]
 [ 0.97551244  0.18871009 -0.11297731]] [ 5.54795074  7.81967545 -4.35967541]
model 1/model 5
RMS(bef) = 52.38
RMS(aft) = 2.08
[[-0.67803943 -0.46776831 -0.56697017]
 [-0.66382056  0.05849731  0.

In [34]:
#Выведем RMSD по всем парам после выравнивания
RMSD_end = 0.0;
for ref_model in structure :
    for alt_model in structure :
        ref_atoms = []
        alt_atoms = []
        for (ref_chain, alt_chain) in zip(ref_model, alt_model) :
            for ref_res, alt_res in zip(ref_chain, alt_chain) :
                assert ref_res.resname == alt_res.resname
                assert ref_res.id      == alt_res.id
                ref_atoms.append(ref_res['CA'].get_coord())
                alt_atoms.append(alt_res['CA'].get_coord())
        sup = Bio.SVDSuperimposer.SVDSuperimposer()
        ref_atoms = np.asarray(ref_atoms)
        alt_atoms = np.asarray(alt_atoms)
        #sup.set(ref_atoms, alt_atoms)
        RMSD_end += sup._rms(ref_atoms, alt_atoms)

In [35]:
#Выводим RMSD до и после выравнивания (попарного)
print(RMSD_start)
print(RMSD_end)

17242.3871893
1324.28253158


In [17]:
#Вывод преобразованных структур в новый файл в том же формате (только вот шапки чет нет, откуда ее брать я не знаю)
io=Bio.PDB.PDBIO()
io.set_structure(structure)
io.save(pdb_out_filename)

In [18]:

    '''
    if ref_model.id == alt_model.id :
        #Check for self/self get zero RMS, zero translation
        #and identity matrix for the rotation.
        #assert numpy.abs(super_imposer.rms) < 0.0000001
        #assert numpy.max(numpy.abs(super_imposer.rotran[1])) < 0.000001
        #assert numpy.max(numpy.abs(super_imposer.rotran[0]) - numpy.identity(3)) < 0.000001
        assert numpy.abs(sup._rms(ref_atoms, alt_atoms)) < 0.0000001
        assert numpy.max(numpy.abs(sup.get_rotran()[1])) < 0.000001
        assert numpy.max(numpy.abs(sup.get_rotran()[0]) - numpy.identity(3)) < 0.000001
    else :
        #Update the structure by moving all the atoms in
        #this model (not just the ones used for the alignment)
        #super_imposer.apply(alt_model.get_atoms())
    '''

IndentationError: unexpected indent (<ipython-input-18-27390619035d>, line 2)